In [8]:
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.models import Sequential
from keras.layers.advanced_activations import LeakyReLU, ELU, PReLU,ThresholdedReLU
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Activation
from keras.utils import np_utils
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import multiprocessing as mp
import datetime
from data_utils import load_CIFAR10
model = Sequential()


In [9]:
#inputs 
cifar10_dir = 'cifar-10-batches-py'
X_train, Y_train, X_test, Y_test = load_CIFAR10(cifar10_dir)
# As a sanity check, we print out the size of the training and test data.
print ('Training data shape: ', X_train.shape)
print ('Training labels shape: ', Y_train.shape)
print ('Test data shape: ', X_test.shape)
print ('Test labels shape: ', Y_test.shape)

Training data shape:  (50000, 32, 32, 3)
Training labels shape:  (50000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [10]:
# Converting to float to be able to perform divison
X_train = X_train.astype('float64')
X_test = X_test.astype('float64')

#Converts class vectors to binary class matrix.
Y_train = np_utils.to_categorical(Y_train, 10)
Y_test = np_utils.to_categorical(Y_test, 10)

#Perform batch normlization on the data
#get the means in order to center the data around zero
mu1 = np.mean(X_train,axis=0)
mu2 = np.mean(X_test,axis=0)

X_train -= mu1
X_test -= mu2
#divide by the standard deviation in order to scale the data 
X_train  /= np.std(X_train,axis=0)
X_test /= np.std(X_test,axis=0)



batch_size = 128
num_classes = 10
epochs = 1000

In [11]:
datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.16,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.16,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False
          ) # randomly flip images

In [12]:
#flatten the input 
model.add(Flatten(input_shape=X_train.shape[1:]))
#construct the layers
model.add(Dense(1200))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.53))

model.add(Dense(750))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.382))

model.add(Dense(250))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))
# using softmax for the output layer against plateu
model.add(Dense(10))
model.add(Activation('softmax'))

In [13]:
ow = datetime.datetime.now().strftime("%I:%M%p on %B %d, %Y")
tensorboard = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)

adam = keras.optimizers.Adam(lr=0.0012, beta_1=0.85, beta_2=0.999, epsilon=1e-08,decay=0.000012)

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

datagen.fit(X_train[:40000]) #training 40000 and validating 10K

checkpointer = ModelCheckpoint(filepath="./Assin2.hdf5", verbose=1, save_best_only=True, monitor='val_acc')

history = model.fit_generator(datagen.flow(X_train[:40000], Y_train[:40000],batch_size=batch_size),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(X_train[40000:], Y_train[40000:]),
                    workers=8, 
                    callbacks=[tensorboard,checkpointer])
model.summary()


Epoch 1/1000
390/390 [==============================] - 10s 27ms/step - loss: 1.9721 - acc: 0.2942 - val_loss: 1.6362 - val_acc: 0.4120

Epoch 00001: val_acc improved from -inf to 0.41200, saving model to ./Assin2.hdf5
Epoch 2/1000
390/390 [==============================] - 10s 26ms/step - loss: 1.7635 - acc: 0.3641 - val_loss: 1.5454 - val_acc: 0.4478

Epoch 00002: val_acc improved from 0.41200 to 0.44780, saving model to ./Assin2.hdf5
Epoch 3/1000
390/390 [==============================] - 10s 26ms/step - loss: 1.6970 - acc: 0.3870 - val_loss: 1.4889 - val_acc: 0.4653

Epoch 00003: val_acc improved from 0.44780 to 0.46530, saving model to ./Assin2.hdf5
Epoch 4/1000
390/390 [==============================] - 10s 26ms/step - loss: 1.6553 - acc: 0.4031 - val_loss: 1.4662 - val_acc: 0.4806

Epoch 00004: val_acc improved from 0.46530 to 0.48060, saving model to ./Assin2.hdf5
Epoch 5/1000
390/390 [==============================] - 13s 33ms/step - loss: 1.6273 - acc: 0.4105 - val_loss: 1.44

390/390 [==============================] - 13s 34ms/step - loss: 1.2630 - acc: 0.5483 - val_loss: 1.0872 - val_acc: 0.6124

Epoch 00083: val_acc did not improve
Epoch 84/1000
390/390 [==============================] - 10s 27ms/step - loss: 1.2660 - acc: 0.5479 - val_loss: 1.0833 - val_acc: 0.6114

Epoch 00084: val_acc did not improve
Epoch 85/1000
390/390 [==============================] - 10s 26ms/step - loss: 1.2717 - acc: 0.5434 - val_loss: 1.0865 - val_acc: 0.6135

Epoch 00085: val_acc did not improve
Epoch 86/1000
390/390 [==============================] - 16s 42ms/step - loss: 1.2643 - acc: 0.5477 - val_loss: 1.0861 - val_acc: 0.6157

Epoch 00086: val_acc improved from 0.61540 to 0.61570, saving model to ./Assin2.hdf5
Epoch 87/1000
390/390 [==============================] - 10s 26ms/step - loss: 1.2645 - acc: 0.5460 - val_loss: 1.0788 - val_acc: 0.6173

Epoch 00087: val_acc improved from 0.61570 to 0.61730, saving model to ./Assin2.hdf5
Epoch 88/1000
390/390 [====================

390/390 [==============================] - 11s 29ms/step - loss: 1.1784 - acc: 0.5769 - val_loss: 1.0177 - val_acc: 0.6431

Epoch 00169: val_acc improved from 0.64210 to 0.64310, saving model to ./Assin2.hdf5
Epoch 170/1000
390/390 [==============================] - 12s 32ms/step - loss: 1.1714 - acc: 0.5804 - val_loss: 1.0123 - val_acc: 0.6410

Epoch 00170: val_acc did not improve
Epoch 171/1000
390/390 [==============================] - 17s 45ms/step - loss: 1.1786 - acc: 0.5753 - val_loss: 1.0166 - val_acc: 0.6414

Epoch 00171: val_acc did not improve
Epoch 172/1000
390/390 [==============================] - 15s 39ms/step - loss: 1.1740 - acc: 0.5778 - val_loss: 1.0106 - val_acc: 0.6455

Epoch 00172: val_acc improved from 0.64310 to 0.64550, saving model to ./Assin2.hdf5
Epoch 173/1000
390/390 [==============================] - 11s 27ms/step - loss: 1.1736 - acc: 0.5829 - val_loss: 1.0190 - val_acc: 0.6400

Epoch 00173: val_acc did not improve
Epoch 174/1000
390/390 [===============

390/390 [==============================] - 11s 28ms/step - loss: 1.1195 - acc: 0.6015 - val_loss: 0.9853 - val_acc: 0.6508

Epoch 00259: val_acc did not improve
Epoch 260/1000
390/390 [==============================] - 11s 27ms/step - loss: 1.1178 - acc: 0.6000 - val_loss: 0.9800 - val_acc: 0.6561

Epoch 00260: val_acc did not improve
Epoch 261/1000
390/390 [==============================] - 12s 31ms/step - loss: 1.1127 - acc: 0.5998 - val_loss: 0.9808 - val_acc: 0.6573

Epoch 00261: val_acc did not improve
Epoch 262/1000
390/390 [==============================] - 10s 26ms/step - loss: 1.1216 - acc: 0.5992 - val_loss: 0.9828 - val_acc: 0.6557

Epoch 00262: val_acc did not improve
Epoch 263/1000
390/390 [==============================] - 16s 40ms/step - loss: 1.1172 - acc: 0.5970 - val_loss: 0.9779 - val_acc: 0.6605

Epoch 00263: val_acc improved from 0.65820 to 0.66050, saving model to ./Assin2.hdf5
Epoch 264/1000
390/390 [==============================] - 10s 26ms/step - loss: 1.1289 

390/390 [==============================] - 10s 27ms/step - loss: 1.0910 - acc: 0.6085 - val_loss: 0.9623 - val_acc: 0.6598

Epoch 00350: val_acc did not improve
Epoch 351/1000
390/390 [==============================] - 10s 26ms/step - loss: 1.0814 - acc: 0.6103 - val_loss: 0.9634 - val_acc: 0.6594

Epoch 00351: val_acc did not improve
Epoch 352/1000
390/390 [==============================] - 10s 26ms/step - loss: 1.0883 - acc: 0.6099 - val_loss: 0.9608 - val_acc: 0.6633

Epoch 00352: val_acc did not improve
Epoch 353/1000
390/390 [==============================] - 10s 25ms/step - loss: 1.0815 - acc: 0.6120 - val_loss: 0.9620 - val_acc: 0.6618

Epoch 00353: val_acc did not improve
Epoch 354/1000
390/390 [==============================] - 10s 27ms/step - loss: 1.0856 - acc: 0.6157 - val_loss: 0.9615 - val_acc: 0.6600

Epoch 00354: val_acc did not improve
Epoch 355/1000
390/390 [==============================] - 11s 29ms/step - loss: 1.0907 - acc: 0.6091 - val_loss: 0.9614 - val_acc: 0.66

...


Epoch 00717: val_acc did not improve
Epoch 718/1000
390/390 [==============================] - 10s 25ms/step - loss: 1.0158 - acc: 0.6321 - val_loss: 0.9165 - val_acc: 0.6810

Epoch 00718: val_acc improved from 0.67980 to 0.68100, saving model to ./Assin2.hdf5
Epoch 719/1000
390/390 [==============================] - 10s 25ms/step - loss: 1.0099 - acc: 0.6391 - val_loss: 0.9170 - val_acc: 0.6788

Epoch 00719: val_acc did not improve
Epoch 720/1000
390/390 [==============================] - 10s 26ms/step - loss: 1.0119 - acc: 0.6372 - val_loss: 0.9154 - val_acc: 0.6784

Epoch 00720: val_acc did not improve
Epoch 721/1000
390/390 [==============================] - 10s 25ms/step - loss: 1.0166 - acc: 0.6349 - val_loss: 0.9186 - val_acc: 0.6805

Epoch 00721: val_acc did not improve
Epoch 722/1000
390/390 [==============================] - 11s 28ms/step - loss: 1.0148 - acc: 0.6376 - val_loss: 0.9185 - val_acc: 0.6786

Epoch 00722: val_acc did not improve
Epoch 723/1000
390/390 [==========

390/390 [==============================] - 12s 31ms/step - loss: 1.0078 - acc: 0.6369 - val_loss: 0.9164 - val_acc: 0.6817

Epoch 00764: val_acc improved from 0.68140 to 0.68170, saving model to ./Assin2.hdf5
Epoch 765/1000
390/390 [==============================] - 11s 28ms/step - loss: 1.0073 - acc: 0.6382 - val_loss: 0.9176 - val_acc: 0.6784

Epoch 00765: val_acc did not improve
Epoch 766/1000
390/390 [==============================] - 10s 25ms/step - loss: 1.0066 - acc: 0.6386 - val_loss: 0.9159 - val_acc: 0.6782

Epoch 00766: val_acc did not improve
Epoch 767/1000
390/390 [==============================] - 10s 26ms/step - loss: 1.0121 - acc: 0.6379 - val_loss: 0.9182 - val_acc: 0.6777

Epoch 00767: val_acc did not improve
Epoch 768/1000
390/390 [==============================] - 11s 27ms/step - loss: 1.0027 - acc: 0.6418 - val_loss: 0.9131 - val_acc: 0.6818

Epoch 00768: val_acc improved from 0.68170 to 0.68180, saving model to ./Assin2.hdf5
Epoch 769/1000
390/390 [===============


Epoch 00808: val_acc did not improve
Epoch 809/1000
390/390 [==============================] - 11s 27ms/step - loss: 0.9997 - acc: 0.6405 - val_loss: 0.9124 - val_acc: 0.6802

Epoch 00809: val_acc did not improve
Epoch 810/1000
390/390 [==============================] - 10s 26ms/step - loss: 1.0090 - acc: 0.6380 - val_loss: 0.9149 - val_acc: 0.6788

Epoch 00810: val_acc did not improve
Epoch 811/1000
390/390 [==============================] - 11s 28ms/step - loss: 0.9993 - acc: 0.6443 - val_loss: 0.9154 - val_acc: 0.6799

Epoch 00811: val_acc did not improve
Epoch 812/1000
390/390 [==============================] - 10s 26ms/step - loss: 0.9991 - acc: 0.6397 - val_loss: 0.9135 - val_acc: 0.6807

Epoch 00812: val_acc did not improve
Epoch 813/1000
390/390 [==============================] - 11s 28ms/step - loss: 0.9979 - acc: 0.6412 - val_loss: 0.9147 - val_acc: 0.6794

Epoch 00813: val_acc did not improve
Epoch 814/1000
390/390 [==============================] - 10s 26ms/step - loss: 1.

390/390 [==============================] - 18s 45ms/step - loss: 0.9933 - acc: 0.6443 - val_loss: 0.9095 - val_acc: 0.6804

Epoch 00901: val_acc did not improve
Epoch 902/1000
390/390 [==============================] - 15s 38ms/step - loss: 0.9938 - acc: 0.6441 - val_loss: 0.9101 - val_acc: 0.6812

Epoch 00902: val_acc did not improve
Epoch 903/1000
390/390 [==============================] - 19s 48ms/step - loss: 0.9907 - acc: 0.6445 - val_loss: 0.9099 - val_acc: 0.6792

Epoch 00903: val_acc did not improve
Epoch 904/1000
390/390 [==============================] - 19s 48ms/step - loss: 0.9904 - acc: 0.6415 - val_loss: 0.9123 - val_acc: 0.6795

Epoch 00904: val_acc did not improve
Epoch 905/1000
390/390 [==============================] - 10s 26ms/step - loss: 0.9915 - acc: 0.6430 - val_loss: 0.9108 - val_acc: 0.6804

Epoch 00905: val_acc did not improve
Epoch 906/1000
390/390 [==============================] - 10s 26ms/step - loss: 0.9923 - acc: 0.6436 - val_loss: 0.9105 - val_acc: 0.68

390/390 [==============================] - 18s 46ms/step - loss: 0.9863 - acc: 0.6462 - val_loss: 0.9051 - val_acc: 0.6843

Epoch 00993: val_acc did not improve
Epoch 994/1000
390/390 [==============================] - 10s 26ms/step - loss: 0.9848 - acc: 0.6475 - val_loss: 0.9068 - val_acc: 0.6819

Epoch 00994: val_acc did not improve
Epoch 995/1000
390/390 [==============================] - 10s 25ms/step - loss: 0.9806 - acc: 0.6456 - val_loss: 0.9057 - val_acc: 0.6825

Epoch 00995: val_acc did not improve
Epoch 996/1000
390/390 [==============================] - 11s 28ms/step - loss: 0.9911 - acc: 0.6449 - val_loss: 0.9049 - val_acc: 0.6835

Epoch 00996: val_acc did not improve
Epoch 997/1000
390/390 [==============================] - 10s 27ms/step - loss: 0.9804 - acc: 0.6478 - val_loss: 0.9054 - val_acc: 0.6832

Epoch 00997: val_acc did not improve
Epoch 998/1000
390/390 [==============================] - 12s 31ms/step - loss: 0.9864 - acc: 0.6453 - val_loss: 0.9070 - val_acc: 0.68

In [28]:
from keras.datasets import cifar10


In [29]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [2]:
plt.figure(1)
plt.plot(history.history['loss'],'r')
plt.plot(history.history['val_loss'],'g')
plt.xticks(np.arange(0, 1000, 2.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train','validation'])


plt.show()
 #Calculating ACCR
score = model.evaluate(x_test, y_test, verbose=1)
print('Test score:', score[0])
print('ACCR:', score[1])



# In[ ]:

# Calculating CCRn
classes = model.predict_classes(x_test, batch_size=batch_size, verbose=1)
y_test = y_test.flatten()
accuracy_per_class = [0.] * 10

for i in range(classes.shape[0]):
    if classes[i] == y_test[i]:
        accuracy_per_class[y_test[i]] += 1
for i in range(10):
    accuracy_per_class[i] /= 1000.0

c = ['airplane', 'automobile','bird','cat','deer','dog','frog','horse','ship','truck']
for i in range(10):
    print("\nCCRn of %s is %f" % (c[i], accuracy_per_class[i]))


# In[ ]:





NameError: name 'plt' is not defined